In [10]:
%run ./common.ipynb


# filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/SOLE/'
filepath='/Users/u1002018/Library/CloudStorage/OneDrive-SharedLibraries-FootLocker/Global Technology Services - DASH Doc Library/AllProjects/'
# filepath='./output/'


datestr = ""

g_Df = pd.DataFrame (columns=['key','FixVersion', 
 'Priority',
 'Acceptance Criteria',
 'Severity Level',
 'Labels',
 'EpicLink',
 'Issuelinks',
 'Status',
 'Components',
 'IssueCreator',
 'creator.emailAddress',
 'creator.displayName',
#  'SubTasks',
 'Reporter.name',
 'reporter.emailAddress',
 'reporter.displayName',
 'issueType',
 'projectKey',
 'projectName',
 'created_ts',
 'updated_ts',
 'ParentLink',
 'summary',
 'StoryPoint',
 'Sprint', 
 'AssigneeName',
 'AssigneeEmailAddress',
 'AssigneeDisplayName',
 'ResolutionDescription',
 'ResolutionName',
 'parent.Summary',
 'parent.key',
 'parent.status',
 'parent.priority',
 'Defect Type',
 'Test Cycle',
 'Bug Severity',
 'Sub-track',
 'RICE Object Type',
 'Complexity',
 'Scope',
 'Baseline Scope',
 'Sprint_state',
 'Sprint_name',
 'Sprint_startDate',
 'Sprint_endDate',
 'Sprint_completeDate',        
 'Sprint_activatedDate',
 'Sprint_sequence',
 'Sprint_goal',
 'Intial SOW',
 'After Global Design',
 'The Rudy Special',
 'Changelog',
 'CT_leadTime',
 'CT_development',
 'CT_discovery',
 'CT_deployment',
 'CT_cancelled',
 'Changelog_Rec'
 ])

pd.set_option('display.max_columns', None)


In [11]:
global logger
logger = setup_logger()
logger.info("=======================================")
logger.info("Script start accumulating data from JIRA")

2024-12-06 17:55:16,044 - root - INFO - =======================================
2024-12-06 17:55:16,045 - root - INFO - Script start accumulating data from JIRA


In [12]:
def data_clean(dataframe):
    if dataframe is not None and not dataframe.empty:
       dataframe= dataframe.filter(['key','fields.fixVersions',
                    'fields.priority.name',
                    'fields.customfield_31601',
                    'fields.customfield_12402.value',
                    'fields.labels',
                    'fields.customfield_10006',
                    # 'fields.issuelinks',
                    'fields.status.name',
                    'fields.components',
                    'fields.creator.name',
                    'fields.creator.emailAddress',
                    'fields.creator.displayName',
                    # 'fields.subtasks',
                    'fields.reporter.name',
                    'fields.reporter.emailAddress',
                    'fields.reporter.displayName',
                    'fields.issuetype.name',
                    'fields.project.key',
                    'fields.project.name',
                    'fields.created',
                    'fields.updated',
                    'fields.customfield_12823',
                    'fields.summary',
                    'fields.customfield_10002',
                    'fields.customfield_10004',
                    'fields.assignee.name',
                    'fields.assignee.emailAddress',
                    'fields.assignee.displayName',
                    'fields.resolution.description',
                    'fields.resolution.name',
                    'fields.parent.fields.summary',
                    'fields.parent.key',
                    'fields.parent.fields.status.name',
                    'fields.parent.fields.priority.name',
                    'fields.customfield_32101.value',
                    'fields.customfield_32302.value',
                    'fields.customfield_11605.value',
                    'fields.customfield_32901',
                    'fields.customfield_16902' ,
                    'fields.customfield_32900.value' ,
                    'fields.customfield_27503' ,
                    'fields.customfield_16902.value',
                    'fields.customfield_33000',
                    'changelog.histories']).rename(columns={'fields.fixVersions': 'FixVersion', 
                                  'fields.priority.name':'Priority',
                                  'fields.customfield_31601': 'Acceptance Criteria',
                                  'fields.customfield_12402.value':'Severity Level',
                                  'fields.labels':'Labels',
                                  'fields.customfield_10006':'EpicLink',
                                #   'fields.issuelinks': 'Issuelinks',
                                  'fields.status.name': 'Status',
                                  'fields.components':'Components',
                                  'fields.creator.name':'IssueCreator',
                                  'fields.creator.emailAddress':'creator.emailAddress',
                                  'fields.creator.displayName': 'creator.displayName',
                                  # 'fields.subtasks':'SubTasks',
                                  'fields.reporter.name':'Reporter.name',
                                  'fields.reporter.emailAddress':'reporter.emailAddress',
                                  'fields.reporter.displayName': 'reporter.displayName',
                                  'fields.issuetype.name':'issueType',
                                  'fields.project.key':'projectKey',
                                  'fields.project.name':'projectName',
                                  'fields.created':'created_ts',
                                  'fields.updated':'updated_ts',
                                  'fields.customfield_12823': 'ParentLink',
                                  'fields.summary':'summary',
                                  'fields.customfield_10002':'StoryPoint',
                                  'fields.customfield_10004':'Sprint',                                  
                                  'fields.assignee.name':'AssigneeName',
                                  'fields.assignee.emailAddress':'AssigneeEmailAddress',
                                  'fields.assignee.displayName':'AssigneeDisplayName',
                                  'fields.resolution.description':'ResolutionDescription',
                                  'fields.resolution.name':'ResolutionName',
                                  'fields.parent.fields.summary': 'parent.Summary',
                                  'fields.parent.key': 'parent.key',
                                  'fields.parent.fields.status.name': 'parent.status',
                                  'fields.parent.fields.priority.name':'parent.priority',
                                  'fields.customfield_32101.value':'Defect Type',
                                  'fields.customfield_32302.value':'Test Cycle',
                                  'fields.customfield_11605.value':'Bug Severity',
                                  'fields.customfield_32901':'Sub-track',
                                  'fields.customfield_32900.value' : 'RICE Object Type',
                                  'fields.customfield_27503' : 'Complexity',
                                  'fields.customfield_16902.value':'Scope',
                                  'fields.customfield_33000':'Baseline Scope',
                                  'changelog.histories':'Changelog'
                                  })
    return dataframe


In [13]:
def fetch( project, issueType):
    global fetch_df 
    logger.info(f"Fetch {issueType}...")
    attribute =' &expand=projects.issuetypes.fields,changelog'
    searchQry=f'?jql= project in (({project})) and issueType="{issueType}"'

    # if cond_flg == True: 
    #     searchQry=f"{searchQry} and status not in (Cancelled, Done)  {attribute}"
    # else:
    searchQry=f"{searchQry} {attribute}"

    fetch_df = getDataSet(searchQry)

    return data_clean(fetch_df)    
    

In [14]:
def extractSprintDetails(l_Df):
    l_Df['Sprint'] = l_Df['Sprint'].apply(lambda x: convert_to_key_value(x))

    l_Df['Sprint_state'] = l_Df['Sprint'].apply(lambda x: sprintvalue(x,'state') if x is not None else "NA" )
    l_Df['Sprint_name']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'name') if x is not None else "NA" )
    l_Df['Sprint_startDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'startDate') if x is not None else "NA" )
    l_Df['Sprint_endDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'endDate') if x is not None else "NA" )
    l_Df['Sprint_completeDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'completeDate') if x is not None else "NA" )
    l_Df['Sprint_activatedDate']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'activatedDate') if x is not None else "NA" )
    # l_Df['Sprint_sequence']= epic_Df['Sprint'].apply(lambda x: sprintvalue(x,'sequence') if x is not None else "NA" )
    l_Df['Sprint_goal']= l_Df['Sprint'].apply(lambda x: sprintvalue(x,'goal') if x is not None else "NA" )
    l_Df= l_Df.drop(columns=['Sprint'])

    return l_Df

In [15]:
def getBaselineLst(element):
   baselinelst =[]
   if element != None:
      if  element != 'NA':
         for idx in element:
            baselinelst.append(idx['value'])
   return baselinelst

In [16]:
def getChangeLog(element):
    changelog_Rec = [] 
    for history in element:
        created_at = history.get('created',None)
        for item in history['items']:
            if item['field'] =='status' :
                changelog_Rec.append({
                    'created_at': created_at,
                    'field': item['field'],
                    'from': item['fromString'],
                    'to': item['toString'],
                })
    return changelog_Rec

In [17]:
def split_list(lst, n):
    """Splits a list into n approximately equal parts."""
    k, m = divmod(len(lst), n)
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [18]:
issueLists = [
            #   {'issueType':"Portfolio Initiative",'partitionCnt':1},
            #   {'issueType':"Product Initiative", 'partitionCnt':1 },
            #   {'issueType':"Epic",'partitionCnt':10},
              {'issueType':"Story",'partitionCnt':80},
            #   {'issueType':"Task",'partitionCnt':10},
            #   {'issueType':"Sub-task",'partitionCnt':10},
            #   {'issueType':"Bug",'partitionCnt':10},l
            #   {'issueType':"Incident",'partitionCnt':10},
            #   {'issueType':"Production Defects",'partitionCnt':10},
            #   {'issueType':"Defect",'partitionCnt':10},
            #   {'issueType':"Issue",'partitionCnt':10},
            #   {'issueType':"Test",'partitionCnt':30}
             ]

strings = projectList()
# strings ='"SOLMerch","SOLEFIN"'
for issueList in issueLists:
    split_string_list = split_list(strings, issueList['partitionCnt'])

    for partition in split_string_list:
        df=fetch(', '.join(['"{}"'.format(value) for value in partition]),issueList['issueType'])
        if df is not None and not df.empty:
            g_Df = pd.concat([g_Df, df ], ignore_index=True)

2024-12-06 17:55:16,291 - root - INFO - Getting report results...
2024-12-06 17:55:16,292 - root - INFO - https://jira.footlocker.com/rest/api/2/project
2024-12-06 17:55:16,838 - root - INFO - <Response [200]>
2024-12-06 17:55:16,839 - root - INFO - Report results received...
2024-12-06 17:55:16,840 - root - INFO - HTTP 200 - OK
2024-12-06 17:55:16,848 - root - INFO - Fetch Story...
2024-12-06 17:55:16,849 - root - INFO - Getting report results...
2024-12-06 17:55:16,849 - root - INFO - https://jira.footlocker.com/rest/api/2/search?jql= project in (("TEST", "CCEP")) and issueType="Story"  &expand=projects.issuetypes.fields,changelog&maxResults=500&startAt=0
2024-12-06 17:56:10,041 - root - INFO - <Response [200]>
2024-12-06 17:56:10,042 - root - INFO - Report results received...
2024-12-06 17:56:10,043 - root - INFO - HTTP 200 - OK
2024-12-06 17:56:10,244 - root - INFO - total: 624
2024-12-06 17:56:10,329 - root - INFO - Getting report results...
2024-12-06 17:56:10,330 - root - INFO -

In [ ]:
###### SOLE PROJECT ###########
# issueLists = [
#               {'issueType':"Portfolio Initiative"},
#               {'issueType':"Product Initiative"},
#               {'issueType':"Epic"},
#               {'issueType':"Story"},
#               {'issueType':"Task"},
#               {'issueType':"Sub-task"},
#               {'issueType':"Bug"},
#               {'issueType':"Incident"},
#               {'issueType':"Production Defects"},
#               {'issueType':"Defect"},
#               {'issueType':"Issue"},
#               {'issueType':"Test"}
#              ]
# projects =["SOLMerch","SOLEFIN"]
# # projects =["SOLMerch"]
# for project in projects:
#     for issueList in issueLists:
#         df=fetch(issueList['issueType'], project)
#         if df is not None and not df.empty:
#             g_Df = pd.concat([g_Df, df ], ignore_index=True)
    

In [ ]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)

In [ ]:
g_Df = extractSprintDetails(g_Df)
g_Df=g_Df.fillna("NA")

g_Df['FixVersion']= g_Df['FixVersion'].apply(lambda x: keyvalue(x,'name') if x is not None else "" )
g_Df['Baseline Scope']=g_Df['Baseline Scope'].apply(lambda x: getBaselineLst(x) )
g_Df['Intial SOW']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 1')] )
g_Df['After Global Design']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 2')] )
g_Df['The Rudy Special']= g_Df['Baseline Scope'].apply(lambda x: {True:'YES',False:'NO'}[x.__contains__('Baseline 3')] )

In [ ]:
# g_Df[g_Df['Intial SOW']=='YES'][['Intial SOW','After Global Design','The Rudy Special','Baseline Scope']]

In [ ]:
g_Df['Changelog_Rec'] = g_Df['Changelog'].apply(lambda x: getChangeLog(x) )

In [ ]:
def format_timedelta(td):
    """Formats a timedelta object to 'day.hours:min:sec' format."""

    seconds = td.total_seconds()
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    return f"{int(days)}.{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

In [ ]:
discovery_state = ["To Do",
            "Backlog",
            "Not Started",
            "Ready",
            "Refinement",
            "Design",
            "New",
            "Drafting",
            "In Review",
            "Pending PO Review",
            "Open",
            "Pending"
            ]
development_state = ['In Progress',
                'On Hold',
                'Rework',
                'Fail',
                'Build',
                'Testing',
                'Reopen',
                'Work In Progress']
deployment_state = ['Ready For Approval',
                'Approved',
                'Done',
                'Delayed',
                'Need Approval',
                'PO Acceptance',
                'Resolved',
                'Closed Complete']

cancel_state = ['Cancelled',
        'Closed - Not Doing',
        'Closed Skipped',
        'Closed Incomplete']

In [ ]:
def calculateCycleTime(l_Df):
    cycleTime = {}
    for index, row in l_Df.iterrows():
        # display(row)
        _status_change = pd.json_normalize(row['Changelog_Rec'])
        if len(_status_change) > 0:
            if (row['Status'] != 'Done'):
                # Get the current time in UTC
                now_utc = datetime.now(pytz.utc)
                # Convert to Eastern Time
                eastern = pytz.timezone('US/Eastern')
                now_est = now_utc.astimezone(eastern)
                new_node = {
                            'created_at': now_est.strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
                             'field': 'status',
                             'from': row['Status'],
                             'to' : 'Active'
                            }
                _status_change.loc[len(_status_change)] = new_node

            newCreate_node = {
                        # 'created_at': row['created_ts'].to_list()[0], 
                            'created_at': row['created_ts'],
                            'field': 'status',
                            'from': 'created',
                            'to' : 'WorkflowState'
                        }
            # _status_change['created_at'] = pd.to_datetime(_status_change['created_at'], format='%Y-%m-%dT%H:%M:%S.%f%z')
           
            # _status_change= _status_change.tolist()
            # _status_change= pd.json_normalize(_status_change[0])
            _status_change.loc[len(_status_change)] = newCreate_node   
            _status_change.sort_values(by=['created_at'],inplace=True)
            _status_change['created_at']= pd.to_datetime(_status_change['created_at'])
            _status_change['time_diff'] = _status_change['created_at'].diff()

            # # _status_change['proj']=proj
            # _status_change = _status_change.reset_index(drop=True)
            # # _status_change.to_csv('./output/transistion_history.csv', mode='a', header=False, index=False)
            # # cycleTime['proj'] = proj
            # # cycleTime['key'] = _status_change['key'][0]
            row['CT_leadTime'] = format_timedelta(_status_change['time_diff'].sum())

            row['CT_discovery'] = format_timedelta(_status_change[_status_change['to'].isin( discovery_state )]['time_diff'].sum())
            
            row['CT_development'] = format_timedelta(_status_change[_status_change['to'].isin( development_state)]['time_diff'].sum())                                        
            row['CT_deployment'] = format_timedelta(_status_change[_status_change['to'].isin(deployment_state)]['time_diff'].sum())
            row['CT_cancelled'] = format_timedelta(_status_change[_status_change['to'].isin(cancel_state )]['time_diff'].sum())   

            # current_status['QA'] = format_timedelta(_status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum())                    
            # display(_status_change)
            # display(row[['key','CT_leadTime', 'CT_development','CT_discovery', 'CT_deployment'  ]])
    l_Df= l_Df.drop(columns=['Changelog'])
    return l_Df


In [ ]:
g_Df = calculateCycleTime(g_Df)

In [ ]:
logger.info("Upload file to sharepoint...")

try:
    g_Df[g_Df['issueType'] == 'Portfolio Initiative'].to_csv(os.path.join(filepath, 'Portfolio_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Product Initiative'].to_csv(os.path.join(filepath, 'Product_Initiative.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Epic'].to_csv(os.path.join(filepath, 'Epic.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Story'].to_csv(os.path.join(filepath, 'Story.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Task'].to_csv(os.path.join(filepath, 'Task.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Sub-task'].to_csv(os.path.join(filepath, 'Subtasks.csv'), index=False)
    g_Df[g_Df['issueType'] == 'Test'].to_csv(os.path.join(filepath, 'Test.csv'), index=False)
    g_Df.loc[g_Df['issueType'].isin(['Bug','Epic','Incident','Production Defects','Defect','Issue'])].to_csv(os.path.join(filepath, 'Bug.csv'), index=False)
except Exception as e:  
    logger.error(f"Failed to upload files to {filepath}: {e}")
    pass


logger.info("Files uploaded")
logger.info("Script ended accumulating data from JIRA")
logger.info("=======================================")

In [ ]:

# 15/* * * * *  /usr/bin/python3 /Users/u1002018/src/PAE/JIRA/py/sole_proj.py

In [ ]:

# def getLinkedIssues(key):
#     check= epic_Df[epic_Df['key'] == key].Issuelinks.astype(bool)
#     if check.to_list()[0] == True:
#         issueLink_str=epic_Df[epic_Df['key'] == key]['Issuelinks']
#         issueLink_str=issueLink_str.to_list()[0][0]

#         searchQry=f'?jql= issue in linkedIssues("{key}")'
#         searchQry_DS = getDataSet(searchQry)
#         linkedIssue=pd.json_normalize(searchQry_DS)

#         specficfields = linkedIssue[['key','fields.status.name','fields.priority.name','fields.summary']]
#         specficfields=specficfields.rename(columns={'fields.status.name': 'status','fields.priority.name':'priority', 'fields.summary':'summary'})
#         json_specficfields =specficfields.to_json(orient='records', lines=True)
#         # print(json_specficfields)
#         return json_specficfields
#     else:
#         return ""
